In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import math

emp_IT=pd.read_csv('C:/Users/dawod/OneDrive/Desktop/Training/estarta/SQL/CSVs/Employee_Profile_IT.csv')
#Cleaning Data
#Fill in missing ages based
emp_IT.fillna({'Age':emp_IT.groupby(['Job Title'])['Age'].transform('mean')},inplace=True)
emp_IT['Age']=emp_IT['Age'].astype(int)

#fill in missing annual salary
emp_IT.fillna({'Annual Salary':emp_IT.groupby(['Job Title'])['Annual Salary'].transform('mean')},inplace=True)

#Fix format of hire date and exit date
emp_IT['Hire Date']=pd.to_datetime(emp_IT['Hire Date'],format='%Y-%m-%d')
emp_IT['Exit Date']=pd.to_datetime(emp_IT['Exit Date'],format='%Y-%m-%d')

#fill in missing hire dates
emp_IT.fillna({'Hire Date':emp_IT.groupby('Age')['Hire Date'].transform('mean')},inplace=True)

#Filling missing ethnicities based on names
emp_IT.loc[emp_IT['EEID'] == 'E00816', 'Ethnicity'] = 'Caucasian'
emp_IT.loc[emp_IT['EEID'] == 'E02855', 'Ethnicity'] = 'Asian'

#Fill in missing Ethnicity and Country based on each other
emp_IT.loc[emp_IT['Country'] == 'Brazil', 'Ethnicity'] = emp_IT.loc[emp_IT['Country'] == 'Brazil', 'Ethnicity'].fillna('Latino')
emp_IT.loc[emp_IT['Country'] == 'China', 'Ethnicity'] = emp_IT.loc[emp_IT['Country'] == 'China', 'Ethnicity'].fillna('Asian')
emp_IT.loc[(emp_IT['Ethnicity'] == 'Black') | (emp_IT['Ethnicity'] == 'Caucasian'),'Country'] = emp_IT.loc[(emp_IT['Ethnicity'] == 'Black') | (emp_IT['Ethnicity'] == 'Caucasian'),'Country'].fillna('United States')

#Fill in missing bonus through ethnicity, gender, and job title
emp_IT.fillna({'Bonus %':emp_IT.groupby(['Ethnicity','Job Title','Gender'])['Bonus %'].transform('mean')},inplace=True)

#Fill in missing city based on country, and ethnicity
emp_IT['City'] = emp_IT['City'].fillna(emp_IT.groupby(['Country', 'Ethnicity'])['City'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan))

#Fill in missing Job Title based on ethnicity and annual salary
emp_IT.loc[emp_IT['EEID'] == 'E04116', 'Job Title'] = 'Director'
#Data cleaned except missing names

#Dropping exit date column
emp_IT.drop(['Exit Date'],axis=1,inplace=True)

#Format exit and hire dates
emp_Exit=pd.read_csv('C:/Users/dawod/OneDrive/Desktop/Training/estarta/SQL/Task2_SQL_and_ML/Task 2/DATA/Employee_Exit.csv')
emp_Exit['Hire Date']=pd.to_datetime(emp_Exit['Hire Date'],format='%m/%d/%Y')
emp_Exit['Exit Date']=pd.to_datetime(emp_Exit['Exit Date'],format='%m/%d/%Y')

#Finding earliest hire date:
f_Date=min(emp_Exit['Hire Date'].min(),emp_IT['Hire Date'].min())
#Convert hire and exit dates into days employed to find the time an employee stayed at the company
emp_Exit['delta_Time']=pd.Series((emp_Exit['Exit Date']-emp_Exit['Hire Date'])/np.timedelta64(1,'D'))
emp_Exit['delta_HT']=pd.Series((emp_Exit['Hire Date']-f_Date)/np.timedelta64(1,'D'))

#Fix formatting of annual salary and bonus % so it can be chagned to float
emp_Exit['Annual Salary']=emp_Exit['Annual Salary'].str.replace('$','')
emp_Exit['Annual Salary']=emp_Exit['Annual Salary'].str.replace(',','')
emp_Exit['Annual Salary']=emp_Exit['Annual Salary'].astype(float)
emp_Exit['Bonus %']=emp_Exit['Bonus %'].str.replace('%','')
emp_Exit['Bonus %']=emp_Exit['Bonus %'].astype(float)

#Removing the outliers:
q1, q3 = np.percentile(emp_Exit['delta_Time'], [25, 75])
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
c_Exit = emp_Exit[(emp_Exit['delta_Time'] >= lower) & (emp_Exit['delta_Time'] <= upper)]

#Setting the X and Y data, taking only the data for IT department since that is the department we are trying to predict the exit dates for
X=c_Exit.loc[c_Exit['Department']=='IT',['Age','Annual Salary','Bonus %','delta_HT']]
y=c_Exit.loc[c_Exit['Department']=='IT',['delta_Time']]

#Seperating training and testing data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=35)

#Train the model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
from sklearn.metrics import mean_absolute_error,mean_squared_error

#Find the mae and rmse
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
rmse = mean_squared_error(y_true=y_test,y_pred=y_pred)

print("MAE:",mae)
print("RMSE:",math.sqrt(rmse))
print("Mean of delta time:",(c_Exit['delta_Time']).mean())

#Fix formatting of annual salary and bonus % so it can be chagned to float

#Predicting exit dates for IT employees
emp_IT['delta_HT']=pd.Series((emp_Exit['Hire Date']-f_Date)/np.timedelta64(1,'D'))
emp_IT['Predicted Exit Date'] = emp_IT['Hire Date'] + pd.to_timedelta(model.predict(emp_IT[['Age','Annual Salary','Bonus %','delta_HT']]).flatten(),unit='D')


#Dropping temporary column
emp_IT.drop(['delta_HT'],axis=1,inplace=True)

# Show the predicted data
#print(emp_IT[['Hire Date','Predicted Exit Date']].to_string())

TypeError: agg function failed [how->mean,dtype->object]